# Tracking Object with Jetbot by Navigating to New Waypoint using GPS

The following code will track Objects using Yolo V5 wile navigating JetBot towarget location and avoiding any obsticals towards the target

### Needed Components:
JetBot
MPU 9250 (For magnetometer)  
Ublox NEO 6M (GPS module)  
Raspberry Pi Camera V2.1

In [ ]:
!ls /dev/tty*

In [ ]:
import sys
import os
import time
import argparse
import numpy as np
import cv2
# from PIL import Image
import tensorrt as trt
import pycuda.driver as cuda
import pycuda.autoinit
from jetbot import Robot
from jetbot import Camera
from tool.utils import *

from absl import app, flags, logging
from absl.flags import FLAGS
from IPython.display import display
from PIL import Image

from Processor import Processor
from Visualizer import Visualizer
robot = Robot()
from IPython.display import clear_output
import Jetson.GPIO as GPIO
GPIO.setmode(GPIO.BOARD)
GPIO.setup(11,GPIO.OUT,initial=GPIO.LOW)
global det_count

GPIO.setup(12,GPIO.OUT,initial=GPIO.LOW)


In [ ]:
import smbus
import math

from imusensor.MPU9250 import MPU9250
from imusensor.filters import kalman

from jetbot import Robot
import PIL

import serial

In [ ]:
# GPS (UART)

ser = serial.Serial("/dev/ttyACM0") # Address of GPS Device connected via USB
gpgga_info = "$GPGGA," # Extract GPGGA line from GPS
GPGGA_buffer = 0
NMEA_buff = 0

# IMU Address
address = 0x68
bus = smbus.SMBus(0)
imu = MPU9250.MPU9250(bus, address)
imu.begin()
imu.loadCalibDataFromFile("docs/calib_real4.json")


### Defining functions for navigation towards the target location

In [ ]:
def convert_to_degrees(raw_value):
    decimal_value = raw_value/100.00
    degrees = int(decimal_value)
    mm_mmmm = (decimal_value - int(decimal_value))/0.6
    position = degrees + mm_mmmm
#     position = "%.4f" %(position)
    return position


# Magneto Meter
#global yaw_prev   
yaw_prev=0#Previous yaw when Object1 was detected
global currTime
currTime = time.time()
sensorfusion = kalman.Kalman()

def imu_yaw():
    global currTime
    imu.readSensor()
    imu.computeOrientation()
    newTime = time.time()
    dt = newTime - currTime
    currTime = newTime

    sensorfusion.computeAndUpdateRollPitchYaw(imu.AccelVals[0], imu.AccelVals[1], imu.AccelVals[2], imu.GyroVals[0], imu.GyroVals[1], imu.GyroVals[2],imu.MagVals[0], imu.MagVals[1], imu.MagVals[2], dt)

#     print("Kalmanroll:{0} KalmanPitch:{1} KalmanYaw:{2} ".format(sensorfusion.roll, sensorfusion.pitch, sensorfusion.yaw))
#     print("KalmanYaw:{0} ".format(sensorfusion.yaw))
    return sensorfusion.yaw
    #time.sleep(0.5) 

def currentHeading(): #Get current Headding/Facing direction from Magnetometer (0 to 360 deg)
    
    imu.readSensor()
    imu.computeOrientation()
    
    magX = imu.MagVals[0] 
    magY = imu.MagVals[1]
    M_PI = math.pi
    offSet = 30 # Correction Angle
    
    if (magY > 0):
        heading = 90 - math.atan(magX/magY)*(180/M_PI)
        
    elif (magY < 0):
        heading = 270 - math.atan(magX/magY)*(180/M_PI)
        
    elif (magY == 0 and magX < 0):
        heading = 180
        
    elif (magY == 0 and magX > 0): 
        heading = 0
        
    heading = heading + offSet
    if heading > 360:
        heading = heading - 360;
    elif heading < 0:
        heading = heading + 360;
        
    return heading


# Navigation

def getWayPoint(currHead, targHead, currLat, currLon, dist): # Get Waypoint when target heading, distance and current location is given
    R = 6378.137 * 1000  # Radius of earth in M 
    
    Ad = dist/R * 180/math.pi # Angular distance
    
    targHead = currHead + targHead # previous targHead is w.r.t North (Bearing Angle)
    
    targLat = np.rad2deg(math.asin(math.sin(np.deg2rad(currLat)) * math.cos(np.deg2rad(Ad)) + math.cos(np.deg2rad(currLat)) * math.sin(np.deg2rad(Ad)) * math.cos(np.deg2rad(targHead)))) 
    targLong = currLon + np.rad2deg(math.atan2(math.sin(np.deg2rad(targHead)) * math.sin(np.deg2rad(Ad)) * math.cos(np.deg2rad(currLat)), math.cos(np.deg2rad(Ad)) - math.sin(np.deg2rad(currLat)) * math.sin(np.deg2rad(targLat))))
    
    return [targLat,targLong]

def getSteering(currHead, currLat, currLon, targLat, targLon): # Steering angle towards target location wrt our current location and heading
    # currHead must be from -360 to 360;
    x = math.cos(np.deg2rad(targLat)) * math.sin(np.deg2rad(currLon-targLon));
    y = math.cos(np.deg2rad(currLat)) * math.sin(np.deg2rad(targLat)) - math.sin(np.deg2rad(currLat))* math.cos(np.deg2rad(targLat)) * math.cos(np.deg2rad(currLon-targLon));
    mag = currHead;
    steering = -np.rad2deg(math.atan2(x,y)) - mag;
    if steering >= 180:
        steering = steering - 360;
    elif steering <= -180:
        steering = steering + 360;
    return steering
        
def getDistance(curLat, curLon, lat2, lon2):  # generally used geo measurement function
    R = 6378.137; # Radius of earth in KM
    dLat = lat2 * math.pi / 180 - curLat * math.pi / 180
    dLon = lon2 * math.pi / 180 - curLon * math.pi / 180
    a = math.sin(dLat/2) * math.sin(dLat/2) + math.cos(curLat * math.pi / 180) * math.cos(lat2 * math.pi / 180) * math.sin(dLon/2) * math.sin(dLon/2)
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1-a))
    d = R * c;
    return d * 1000; # meters

def mapf(x, in_min, in_max, out_min, out_max):
    return (x - in_min) * (out_max - out_min) / (in_max - in_min) + out_min

def differentialDrive(steeringAngle, driveSpeed):
    # steeringAngle must be from -180 to 180.

    leftDriveSpeed = driveSpeed;
    rightDriveSpeed = driveSpeed;
    if (steeringAngle <= 0) and (steeringAngle > -90):
        L = leftDriveSpeed - mapf(abs(steeringAngle), 0,180, 0,driveSpeed) # slow left down
        R = rightDriveSpeed
        
    if (steeringAngle < -90) and (steeringAngle >= -180):
        L = leftDriveSpeed - 2*mapf(abs(steeringAngle), 0,180, 0,driveSpeed) # slow left down
        R = rightDriveSpeed
      
    
    if (steeringAngle > 0) and steeringAngle <= 90:
        L = leftDriveSpeed
        R = rightDriveSpeed - mapf(abs(steeringAngle), 0,180, 0,driveSpeed) # slow right down
    
    if (steeringAngle > 90) and steeringAngle <= 180:
        L = leftDriveSpeed
        R = rightDriveSpeed - 2*mapf(abs(steeringAngle), 0,180, 0,driveSpeed) # slow right down
        
    L_R_velocity = [L, R,mapf(abs(steeringAngle), 0,180, 0,driveSpeed)]
    return L_R_velocity

# robot.stop

## Go towards the target location

Potential errors would be:
"ValueError: could not convert string to float:"  
Then wait untill GPS locks to the satilite

In [ ]:
Trig = 1
targ__Distance = 20
targ__Head = 10
def goToTargetWRTdist(Speed,targHead,targDistance): # Going to target location based on the robot's location (You may uses it if u dont know the target lat and long)

    imu.readSensor() # Read IMU Sensor
    imu.computeOrientation()

    received_data = (str)(ser.readline()) #read NMEA string received
    GPGGA_data_available = received_data.find(gpgga_info)   #check for NMEA GPGGA string

    if (GPGGA_data_available>0): # when GPS is active
        GPGGA_buffer = received_data.split("$GPGGA,",1)[1]  #store data coming after “$GPGGA,” string
        NMEA_buff = (GPGGA_buffer.split(','))
        nmea_time = []
        nmea_latitude = []
        nmea_longitude = []
        nmea_time = float(NMEA_buff[0]) + 53000                    #extract time from GPGGA string
        nmea_latitude = NMEA_buff[1]                #extract latitude from GPGGA string
        nmea_latitude_dir = NMEA_buff[2]                #Lat direction
        nmea_longitude = NMEA_buff[3]               #extract longitude from GPGGA string
        nmea_long_dir = NMEA_buff[4]                #Long direction  

        GPS_Fix = NMEA_buff[5]
        satellite_count = NMEA_buff[6]
        Alt = NMEA_buff[8]

        lat = float(nmea_latitude)
        lat = convert_to_degrees(lat)
        longi = float(nmea_longitude)
        longi = convert_to_degrees(longi)

        currentHeading1 = currentHeading()
        print("currentHeading:",currentHeading1)
        currentLat = lat
        currentLon = longi

        print('Time: ',nmea_time)
        print('Current Location: ',currentLat,',',currentLon)

        driveSpeed = Speed
        
        if Trig == 1:
            [targetLat,targetLon] = getWayPoint(currentHeading1, targ__Head, currentLat, currentLon, targ__Distance)
        Trig = 0

        steerAng = getSteering(currentHeading1, currentLat, currentLon, targetLat, targetLon)
        distFromTraget = getDistance(currentLat, currentLon, targetLat, targetLon)

        print('Steering Angle: ',steerAng)
        print('Distance from Target: ',distFromTraget)
        
        if distFromTraget > 3:
            drive = differentialDrive(steerAng, driveSpeed)
            LWSpeed = drive[0]
            RWSpeed = drive[1]
            print('Speed:\t',LWSpeed,'\t,',RWSpeed)
            robot.set_motors(LWSpeed,RWSpeed)
#             clear_output(wait=True)
            
        else:
            print("Goal Reached")
            robot.set_motors(0,0)

In [ ]:
def goToTarget(Speed,targetLat,targetLon): # Go to target location based on your input lat and lon

    imu.readSensor() # Read IMU Sensor
    imu.computeOrientation()

    received_data = (str)(ser.readline()) #read NMEA string received
    GPGGA_data_available = received_data.find(gpgga_info)   #check for NMEA GPGGA string

    if (GPGGA_data_available>0): # when GPS is active
        GPGGA_buffer = received_data.split("$GPGGA,",1)[1]  #store data coming after “$GPGGA,” string
        NMEA_buff = (GPGGA_buffer.split(','))
        nmea_time = []
        nmea_latitude = []
        nmea_longitude = []
        nmea_time = float(NMEA_buff[0]) + 53000                    #extract time from GPGGA string
        nmea_latitude = NMEA_buff[1]                #extract latitude from GPGGA string
        nmea_latitude_dir = NMEA_buff[2]                #Lat direction
        nmea_longitude = NMEA_buff[3]               #extract longitude from GPGGA string
        nmea_long_dir = NMEA_buff[4]                #Long direction  

        GPS_Fix = NMEA_buff[5]
        satellite_count = NMEA_buff[6]
        Alt = NMEA_buff[8]

        lat = float(nmea_latitude)
        lat = convert_to_degrees(lat)
        longi = float(nmea_longitude)
        longi = convert_to_degrees(longi)

        currentHeading1 = currentHeading()
        print("currentHeading:",currentHeading1)
        currentLat = lat
        currentLon = longi

        print('Time: ',nmea_time)
        print('Current Location: ',currentLat,',',currentLon)

        driveSpeed = Speed

        steerAng = getSteering(currentHeading1, currentLat, currentLon, targetLat, targetLon)
        distFromTraget = getDistance(currentLat, currentLon, targetLat, targetLon)

        print('Steering Angle: ',steerAng)
        print('Distance from Target: ',distFromTraget)
        
        

        if distFromTraget > 3:
            drive = differentialDrive(steerAng, driveSpeed)
            LWSpeed = drive[0]
            RWSpeed = drive[1]
            print('Speed:\t',LWSpeed,'\t,',RWSpeed)
            robot.set_motors(LWSpeed,RWSpeed)
#             clear_output(wait=True)
            
        else:
            print("Location Reached")
            robot.set_motors(0,0)

## Defining Collision Avoidance
Inputs needed: Latitude and Longitude of the traget loctaion

In [ ]:

BOUNDARY_TH=0.2# Area Threshold for boundary restriction of Object1
BOUNDARY_TH_2=0.05# Area threshold for Object2
PREV_MOVE='' # Previous move
#opt = parser.parse_args()

HEADING_SPEED=0.18

targLat = <specify lat>
targLon = <specify long>

#Function for collision avoidance

def col_avoid(img,frame_id,lft,THRESH=60,row_start=470,DARK_TH=700): # THRESH=80,row_start=470,DARK_TH=1500
    img_gray=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    #ret,th=cv2.threshold(img_gray,5,255,cv2.THRESH_BINARY)
    img_gray[img_gray>THRESH]=255
    img_gray[img_gray!=255]=0
    th=img_gray
    cv2.imwrite('Col-output/cam-out_'+str(frame_id)+'.jpg',th)
    
    CEN_TH=DARK_TH
    STBD_TH=DARK_TH
    PORT_TH=DARK_TH

    th_new=th[row_start::,100:500]
    th_port=th[row_start::,0:100]
    th_stbd=th[row_start::,500::]
    # Area in the respective zone need to be subtracted by the non zero pixles which gives dark zone
    dark_area=th_new.shape[0]*th_new.shape[1]-cv2.countNonZero(th_new)
    port_area=th_port.shape[0]*th_port.shape[1]-cv2.countNonZero(th_port)
    stbd_area=th_stbd.shape[0]*th_stbd.shape[1]-cv2.countNonZero(th_stbd)
#     print(f'port atea:{port_area},dark area:{dark_area},stbd area{stbd_area}')

        
#     if frame_id%3==0 and frame_id>1:
    if dark_area<CEN_TH:
        print('forward path clear',dark_area)
        goToTarget(HEADING_SPEED,targLat,targLon)
#             robot.set_motors(0.3, 0.3-0.05)
#         time.sleep(0.5)
#             robot.stop()
    elif port_area<PORT_TH or stbd_area<STBD_TH:
        if port_area>stbd_area:
            print('Dark zone.. moving RIGHT',dark_area)
            robot.right(0.14)
            time.sleep(0.5)
            robot.stop()
        else:
            print('Dark zone.. moving LEFT',dark_area)
            robot.left(0.15)
            time.sleep(0.5)
            robot.stop()
    else:
        print('Dark zone.. moving BACK',dark_area)
        robot.backward(HEADING_SPEED)
        time.sleep(0.5)
        robot.stop()
        robot.left(0.15)
        time.sleep(0.5)
        robot.stop()

            
#             time.sleep(0.5)
#             robot.stop()
#             mov_circle_b(0.4,delay=1)
#     #elif port_area<PORT_TH or stbd_area<STBD_TH:
    
    
#     elif lft==1:                    #elif stbd_area<STBD_TH and lft==1:
#         #print('obstacle port side')
#         robot.right(0.14)
#         time.sleep(0.5)
#         robot.stop()
#         lft=0
#     elif lft==0:#elif port_area<PORT_TH and lft==0:
#         #print('obstacle stbd side')
#         robot.left(0.15)
#         time.sleep(0.5)
#         robot.stop()
#         lft=1
        
    return lft



In [ ]:
# gpsWyolov5_migm2
# yolov5_inference-migm2

In [ ]:
#Actual code
try:
    # Sometimes python2 does not understand FileNotFoundError
    FileNotFoundError
except NameError:
    FileNotFoundError = IOError

def GiB(val):
    return val * 1 << 30


In [ ]:

def box_position(boxes,classes,robot,class_names):
    areas={}
    cen={}
    pos={}
    box_co={}
    h,w =640.0,640.0
    for cl in class_names:
        areas[cl]=0.0
        cen[cl]=0.0
        pos[cl]=''
        box_co[cl]=[0.0,0.0,0.0,0.0]
        
    for box,cls in zip(boxes, classes):
        x1, y1, x2, y2 = box
        c=(x1+x2)/2.0
        box_area_per=(x2-x1)*(y2-y1)/(h*w)
        areas[class_names[cls]]=box_area_per
        cen[class_names[cls]]=c 
        box_co[class_names[cls]]=x1, y1, x2, y2 
        
        if c<(0.3*w):
            box_pos='P'
        elif c>(0.7*w):
            box_pos='S'
        else:
            box_pos='C'
        pos[class_names[cls]]=box_pos
            
    return areas,cen,pos,box_co

def box_mov(area_per,cen,box_pos):
    global PREV_MOVE
    #x1, y1, x2, y2 = box
    h,w =640.0,640.0
    cmnd=0.0
    
    if box_pos=='P' and area_per<BOUNDARY_TH:
        cen_t=cen/w
        #print('Area % is:',0.31-cen_t)
        if abs(0.28-cen_t)>0.2:
            cmnd=0.2
            #robot.left(0.25)
        elif 0.28-cen_t>0.18:
            cmnd=0.25-cen_t
            #robot.left(0.28-cen_t)
        else:
            cmnd=0.28-cen_t
        if cmnd<0.15:
            cmnd=0.14
        robot.left(cmnd)
        time.sleep(0.4)
        robot.stop()
        PREV_MOVE='L'
        print(f'{box_pos} command:{cmnd}')
    elif box_pos=='S' and area_per<BOUNDARY_TH:
        #robot.right(0.3)#
        cen_t=cen/w
        if abs(cen_t-0.78)>0.22:
            cmnd=0.19
            #robot.right(0.25)
        elif cen_t-0.78>0.18:
            cmnd=cen_t-0.78
            #robot.right((cen/w)-0.78)
        else:
            cmnd=cen_t-0.74
        if cmnd<0.15:
            cmnd=0.13
        robot.right(cmnd)
        time.sleep(0.4)
        robot.stop()
        PREV_MOVE='R'
        print(f'{box_pos} command:{cmnd}')
        
    elif box_pos=='C' and area_per<BOUNDARY_TH:
        robot.set_motors(0.18, 0.18)
        time.sleep(1)
        robot.stop()
        PREV_MOVE='F'
    

In [ ]:
# class_names=['Object1','Object2']
# # location={'Object1':'','Object2':''}
# # location[class_names[0]]='S'
# # location
# df={}
# df[class_names[0]]=0
# df[class_names[1]]=1
# df

### Movement Def

In [ ]:
def mov_forward(val,delay=0.5):
    robot.set_motors(val, val-0.05)
    #robot.forward(val)
    time.sleep(delay)
    robot.stop()
    #print('Moving forward..')

def mov_circle(val,delay=0.5):
    robot.set_motors(val, val-0.13)
    #robot.forward(val)
    time.sleep(delay)
    robot.stop()
    #print('Moving forward..')
def mov_circle_l(val,delay=0.5):
    robot.set_motors(val-0.13, val)
    #robot.forward(val)
    time.sleep(delay)
    robot.stop()
def mov_circle_b(val,delay=0.5):
    robot.set_motors(-val, -val+0.13)
    #robot.forward(val)
    time.sleep(delay)
    robot.stop()
    #print('Moving forward..')
    
def mov_left(val,delay=0.5):
    robot.left(val)
    time.sleep(delay)
    robot.stop()
def mov_right(val,delay=0.5):
    robot.right(val)
    time.sleep(delay)
    robot.stop()
def mov_back(val,delay=0.5):
    robot.backward(val)
    time.sleep(delay)
    robot.stop()
def spiral_search(frame_id):
    if frame_id%20 ==0 and frame_id>1:
        print('Moving in circle')
        mov_circle(0.5,delay=1.28)
    else:
        mov_forward(0.4,0.5)
def st_search(frame_id,lft):
    global PREV_MOVE
    if frame_id%3==0 and frame_id>1:
        mov_forward(0.4,0.4)
        PREV_MOVE='F'
        print('Moving forward')
    else:
        if lft==0:
            mov_left(0.15,0.5)
            time.sleep(0.2)
            lft=1
            PREV_MOVE='L'
        else:
            mov_right(0.14,0.5)
            time.sleep(0.2)
            lft=0
            PREV_MOVE='R'
    return lft

def st_search_new(frame_id,lft):
    if lft==1 or (frame_id%5==0 and frame_id>1):
        if lft==0:
            mov_left(0.2,0.3)
            lft=1
        else:
            mov_right(0.3,0.4)
            lft=0
    else:
        mov_forward(0.4,0.5)
        print('Moving forward')
        
    return lft

def revert_prev_move():
    global PREV_MOVE
    if PREV_MOVE=='L':
        PREV_MOVE='R'
        mov_right(0.14,0.5)
        time.sleep(0.2)
        
    elif PREV_MOVE=='R':
        PREV_MOVE='L'
        mov_left(0.15,0.5)
        time.sleep(0.2)
        
    elif PREV_MOVE=='F':
        PREV_MOVE='B'
        mov_forward(-0.4,0.4)
        


In [ ]:
def main(engine_path, image_size,opt,processor,visualizer,camera):
    ##To create camera instance 
    
    det_count=0  #count of consecutive detections
    Object1_bound=0 # Object1 boundary reached flag
    Object2_bound=0 # Object2 boundary reached flag
    Object1_search_count=0
    conf=opt['conf']
    input_size = opt['img_size']#FLAGS.size

    frame_id = 0
    
    lft=0
    image_num = 100000
    #fps = FPS().start()
    try:
        while True:
            image = camera.value #change['new']
            frame_size = image.shape[:2]

            # Press Q on keyboard to  exit
            if cv2.waitKey(25) & 0xFF == ord('q'):
                camera.stop()

            image_src = image #cv2.imread(image_path)
            num_classes = opt['classes']   # mahesh  from 80
            namesfile = opt['name']   # names file
            class_names = load_class_names(namesfile)

            img = cv2.resize(image_src, (opt['img_size'], opt['img_size']))
            output = processor.detect(img) 
            boxes, confs, classes = processor.post_process(output,origin_w=opt['img_size'],origin_h=opt['img_size'], conf_thres=conf)
            
            areas,cen,pos,box_co=box_position(boxes,classes,robot,class_names)
            #print('Center positions:',cen['Object1']/640,cen['Object2']/640)
            #print(areas)
            if areas['Object1']>0 and Object1_bound==0:
                GPIO.output(11,GPIO.HIGH)
                det_count+=1
#                 if areas['Object2']>0 and cen['Object2']>(box_co['Object1'][0]-10) and cen['Object2']<(box_co['Object1'][2]+10):
#                     GPIO.output(12,GPIO.HIGH)
#                     print('Object2 is inside Object1')
#                     if areas['Object2']<BOUNDARY_TH_2:
#                         box_mov(areas['Object2'],cen['Object2'],pos['Object2'])
#                     else:
#                         print('destination reached')
#                 else:
#                     GPIO.output(12,GPIO.LOW)
                yaw_prev=imu_yaw()  #computing the yaw

                if areas['Object1']<BOUNDARY_TH:
                    print('Moving towards Object1...')
                    box_mov(areas['Object1'],cen['Object1'],pos['Object1'])
                else:
                    print('Object1 boundary reached..')
                    Object1_bound=1
            elif Object1_bound==1:
                if areas['Object2']>0:
                    if areas['Object2']<BOUNDARY_TH_2 and sen_bound==0:
                        box_mov(areas['Object2'],cen['Object2'],pos['Object2'])
                    else:
                        robot.set_motors(0,0)
                        print('Final destination reached..')
                        sen_bound=1
                    GPIO.output(12,GPIO.HIGH)
                else:
#                     GPIO.output(12,GPIO.LOW)
#                     print('Searching for Object2')
#                     Object1_search_count+=1
                    print('Object1 boundary reached..')
                    sen_bound=1
                    robot.set_motors(0,0)
                    print('Final destination reached..')


            elif det_count>0:
                yaw_present=imu_yaw()
                yaw_diff=yaw_prev-yaw_present
                if abs(yaw_diff)>0.5:
                    if yaw_diff<0:
                        mov_right(0.15,0.5)
                        time.sleep(0.15)
                    else:
                        mov_left(0.15,0.5)
                        time.sleep(0.15)
                        
#                     differentialDrive(yaw_diff,0.15)
#                     time.sleep(0.15)
            else:
                det_count=0 if det_count<=0 else det_count-1     
                GPIO.output(11,GPIO.LOW)
                GPIO.output(12,GPIO.LOW)
                if opt['search']=='spiral':
                    spiral_search(frame_id)       #spiral search
                else:
                    if opt['col_avoid']=='True':
                        lft=col_avoid(img,frame_id,lft,THRESH=60,row_start=470)
                    else:
                        lft=st_search(frame_id,lft)  #standard search
            if opt['show_out']=='True':
                retimg=visualizer.draw_results(img, boxes, confs, classes,class_names,BOUNDARY_TH)
                retimg=cv2.cvtColor(retimg,cv2.COLOR_BGR2RGB)
                display(Image.fromarray(retimg))
                time.sleep(0.5)
                clear_output(True)
            if opt['save']=='True':
                retimg=visualizer.draw_results(img, boxes, confs, classes,class_names,BOUNDARY_TH)
                cv2.imwrite('det-output/det-YoloImg_'+str(image_num)+'.jpg',retimg)
                image_num = image_num + 1
#             if save_out==True:
#                 out.write(retimg)

            frame_id += 1
            #print("Elasped time: {:.2f}".format(fps.elapsed()))
            #print("FPS: {:.2f}".format(fps.fps()))

        #fps.stop()
    except KeyboardInterrupt:
        GPIO.output(11,GPIO.LOW)
        GPIO.output(12,GPIO.LOW)
        print('keyboard interrupt & stopping the camera')
        robot.stop()
        camera.stop()
        pass 


## Starting
Note: Robot will start moving

In [ ]:


opt={'eng':'(Location of ".engine file")','classes':2,'conf':0.3,'img_size':640,'name':'data/migm.names','save':'True','show_out':'True','search':'True','col_avoid':'True'} 
#print(opt.weights)
engine_path = opt['eng']
image_size=((opt['img_size']),(opt['img_size']))
print('Engine loading')
processor = Processor(engine_path,opt['classes'])
print('Engine loaded....')
visualizer = Visualizer()
camera = Camera.instance(width=640, height=640)
main(engine_path, image_size,opt,processor,visualizer,camera) #Starting Main Function

In [ ]:
camera.stop()
GPIO.output(11,GPIO.LOW)
GPIO.output(12,GPIO.LOW)
# print('keyboard interrupt & stopping the camera')
# robot.stop()
# camera.stop()